# Multi-Agent Routing with LangGraph

This notebook demonstrates how to build a multi-agent system that directs queries to specialized language models (LLMs) based on their content. We will use LangGraph to create a stateful graph that manages the routing logic.

## Key Features of the Router

-  **Router Functionality:** Enables an LLM to select a single step from multiple options.
-  **Router Control:** Offers limited control, as the LLM makes one decision and generates specific output from predefined choices.

## Overview of Graph Structure

-  A code-specialized LLM
-  A math-specialized LLM
-  A creative-specialized LLM
-  A general-purpose LLM

## 1. Imports and State Definition

In [ ]:
from typing import TypedDict, Literal
from langgraph.graph import StateGraph, END
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage
from rich import print
from langchain_ollama import ChatOllama, OllamaEmbeddings

load_dotenv()

In [ ]:
# Define the state structure
class AgentState(TypedDict):
    query: str
    route: str
    response: str
    reasoning: str

## 2. Mock LLM Definitions

To simulate a multi-agent environment without requiring actual API calls, we define several mock LLM functions. Each function represents a specialized agent and returns a formatted string indicating which LLM was called.

In a real-world application, these would be replaced with actual calls to different LLM APIs or models.

In [ ]:
# Mock LLM responses (replace with actual LLM calls)
def code_llm(query: str) -> str:
    """Specialized LLM for coding questions"""
    return f"[CODE LLM] Here's a programming solution for: {query}"

def math_llm(query: str) -> str:
    """Specialized LLM for math questions"""
    return f'[MATH LLM] Mathematical analysis of: {query}'

def general_llm(query: str) -> str:
    """General purpose LLM"""
    return f'[GENERAL LLM] General response to: {query}'

def creative_llm(query: str) -> str:
    """Specialized LLM for creative tasks"""
    return f'[CREATIVE LLM] Creative response to: {query}'

## 3. The Router Function

The `route_query` function is the core of our routing logic. It inspects the user's query for specific keywords to determine the most appropriate LLM. 

It uses predefined lists of keywords for different domains (code, math, creative). If a keyword is found, it sets the `route` and `reasoning` fields in the agent's state. If no specific keywords are matched, it defaults to the 'general' route.

In [ ]:
def route_query(state: AgentState) -> AgentState:
    """Analyze query and determine which LLM to use."""
    # Normalize the query to lowercase for case-insensitive matching
    query = state['query'].lower()

    # Define keywords for identifying code-related queries
    code_keywords = [
        'python', 'javascript', 'code', 'function', 'programming', 
        'algorithm', 'debug', 'api', 'class', 'variable',
    ]

    # Define keywords for identifying math-related queries
    math_keywords = [
        'calculate', 'equation', 'math', 'formula', 'solve', 
        'probability', 'statistics', 'derivative', 'integral',
    ]

    # Define keywords for identifying creative writing queries
    creative_keywords = ['story', 'poem', 'creative', 'write', 'imagine', 'fictional', 'character', 'plot', 'narrative']

    # Determine the appropriate route by matching keywords in the query
    if any(keyword in query for keyword in code_keywords):
        route = 'code'
        reasoning = 'Detected programming/coding related query'
    # Check for math keywords only if no code keywords were found
    elif any(keyword in query for keyword in math_keywords):
        route = 'math'
        reasoning = 'Detected mathematical/calculation query'
    # Check for creative writing keywords only if no code or math keywords were found
    elif any(keyword in query for keyword in creative_keywords):
        route = 'creative'
        reasoning = 'Detected creative writing query'
    else:
        # Default to the general model if no specific keywords are found
        route = 'general'
        reasoning = 'No specific specialization detected, using general LLM'

    # Update the agent's state with the routing decision and justification
    state['route'] = route
    state['reasoning'] = reasoning
    
    # Return the modified state object
    return state


In [ ]:
# Enhanced router: Use an LLM to classify the query into one of the target domains.
# Falls back to keyword routing (defined below) if LLM unavailable or errors.

def llm_route_query(state: AgentState) -> AgentState:
    
    query = state["query"].strip()

    # llm = ChatOpenAI(model="gpt-4.1-nano", max_tokens=500)
    llm = ChatOllama(model='llama3.2', temperature=0)
    
    system = SystemMessage(content="""
You are a routing classifier. Given a user query, respond ONLY with one token from this set:
code | math | creative | general
Definitions:
- code: programming, software engineering, APIs, debugging, algorithms.
- math: equations, calculus, probability, statistics, numeric problem solving.
- creative: storytelling, poems, fiction, characters, plot, imaginative writing.
- general: anything else (explanations, science, history, general knowledge).
If ambiguous, choose the most plausible specialized category else 'general'.
Return a JSON object: {"route": <one of above>, "reasoning": "short explanation"}.
Strict JSON.
""".strip())
    human = HumanMessage(content=f"Query: {query}")

    raw = llm.invoke([system, human])
    text = raw.content if hasattr(raw, 'content') else str(raw)
    # Attempt to parse JSON
    route = "general"
    reasoning = "LLM routing fallback to general"
    import json
    m = None
    try:
        data = json.loads(text)
        route = data.get("route", route)
        reasoning = data.get("reasoning", reasoning)
    except json.JSONDecodeError:
        # Heuristic extraction
        lowered = text.lower()
        for candidate in ["code", "math", "creative", "general"]:
            if candidate in lowered:
                route = candidate
                reasoning = f"Heuristic parse from LLM output: {text[:60]}"
                break
    state["route"] = route
    state["reasoning"] = reasoning
    return state
    

## 4. Agent Nodes

Each of these functions represents a node in our graph. When the graph transitions to one of these nodes, the corresponding function is executed. Each function calls its specialized mock LLM with the query, and updates the `response` field in the state.

In [ ]:
def handle_code_query(state: AgentState) -> AgentState:
    """Process query with code-specialized LLM"""
    response = code_llm(state['query'])
    state['response'] = response
    return state

def handle_math_query(state: AgentState) -> AgentState:
    """Process query with math-specialized LLM"""
    response = math_llm(state['query'])
    state['response'] = response
    return state

def handle_creative_query(state: AgentState) -> AgentState:
    """Process query with creative-specialized LLM"""
    response = creative_llm(state['query'])
    state['response'] = response
    return state

def handle_general_query(state: AgentState) -> AgentState:
    """Process query with general LLM"""
    response = general_llm(state['query'])
    state['response'] = response
    return state

## 5. Graph Construction

Now we build the graph using `StateGraph`.

1.  **Nodes**: We add the `router` and each of the specialized handler functions as nodes in the graph.
2.  **Entry Point**: We set the `router` node as the entry point. All queries will start here.
3.  **Conditional Edges**: After the `router` node, we use `add_conditional_edges`. The `determine_next_node` function reads the `route` from the state and tells the graph which specialized node to go to next.
4.  **End Points**: After each specialized node has done its work, it transitions to the `END` state, finishing the execution for that query.
5.  **Compilation**: Finally, we compile the workflow into a runnable application.

In [ ]:
def determine_next_node(state: AgentState) -> Literal['code', 'math', 'creative', 'general']:
    """Return the next node based on routing decision"""
    return state['route']

def create_routing_agent(use_llm: bool = True):
    """Create and return the LangGraph routing agent.

    Args:
        use_llm: If True and LLM available, use LLM router; otherwise keyword router.
    """

    workflow = StateGraph(AgentState)

    # Choose router function
    router_fn = llm_route_query if use_llm and ChatOpenAI else route_query

    # Add nodes
    workflow.add_node('router', router_fn)
    workflow.add_node('code', handle_code_query)
    workflow.add_node('math', handle_math_query)
    workflow.add_node('creative', handle_creative_query)
    workflow.add_node('general', handle_general_query)

    workflow.set_entry_point('router')

    workflow.add_conditional_edges(
        'router', determine_next_node, {'code': 'code', 'math': 'math', 'creative': 'creative', 'general': 'general'}
    )

    workflow.add_edge('code', END)
    workflow.add_edge('math', END)
    workflow.add_edge('creative', END)
    workflow.add_edge('general', END)

    return workflow.compile()

## 6. Execution and Demonstration

Below we instantiate the routing agent. If an OpenAI-compatible API key is present (OPENAI_API_KEY), the router uses an LLM to classify the query; otherwise it falls back to deterministic keyword routing. You can override the model via the `ROUTER_MODEL` environment variable. Set `use_llm=False` in `create_routing_agent` to force keyword routing.


In [ ]:
agent = create_routing_agent(use_llm=True)
print('Subgraph structure (router uses LLM =', bool(ChatOpenAI), ')')
# Optional: Display a visualization of the graph's structure.
try:
    from IPython.display import Image, display
    display(Image(agent.get_graph().draw_mermaid_png()))
except Exception:
    pass

In [ ]:
from rich.panel import Panel
from rich.rule import Rule

test_queries = [
    'How do I implement a binary search algorithm in Python?',
    "What's the integral of 2x³ + 5x² - 3x + 7?",
    'Write a mysterious short story about a lighthouse keeper',
    'What causes the northern lights phenomenon?',
    'Help me optimize this SQL query performance',
    'Solve for x: 3x² - 12x + 9 = 0',
]

print(Panel.fit("[bold green]LangGraph LLM-Powered Routing Agent Demo[/bold green]", style="bold"))

for i, query in enumerate(test_queries, 1):
    print(Rule(f"[bold]Query {i}[/bold]", style="red"))
    print(f"[bold]Query:[/bold] [italic]{query}[/italic]\n")

    # Run the agent
    result = agent.invoke({'query': query})

    # Print results with rich formatting
    print(f"   [bold cyan]Route:[/bold cyan] [yellow]{result['route']}[/yellow]")
    print(f"   [bold cyan]Reasoning:[/bold cyan] {result['reasoning']}")
    print(f"   [bold cyan]Response:[/bold cyan] {result['response'][:100]}...")